In [1]:
import json
import requests
import time
import multiprocessing as mp
from qualtrics_functions import *
import pandas as pd
import numpy as np
from io import StringIO
import base64
from markdown import markdown
from datetime import datetime

In [2]:
# ---- create qualtrics IDs
qualtrics_ids = create_qualtrics_ids(data_center = 'duke.ca1.qualtrics.com',
                                     api_token = 'e2b6axpAfVIEd7m4R7mgdDa6nRiXtOR9bzhY1uHC')

# ---- get "user data"
user_data = pd.read_excel('user_data.xlsx', engine='openpyxl')

In [3]:
delete_survey(qualtrics_ids, survey = get_most_recent_survey(qualtrics_ids))

retrieved "recruiting_built_from_api" (id = SV_246nbvy1UoYBUh0)
survey SV_246nbvy1UoYBUh0 deleted


In [4]:
# --------------------------------------------------------------------------------
# ---- create survey on Qualtrics ------------------------------------------------
# --------------------------------------------------------------------------------
survey = create_survey(qualtrics_ids, survey_name = 'recruiting_built_from_api')

# --------------------------------------------------------------------------------
# ---- create survey questions ---------------------------------------------------
# --------------------------------------------------------------------------------
survey_q = []

# ---- welcome
survey_q.append(
create_text_block(question_number = 1, 
                  question_text = markdown('''Thank you so much for participating in our research.  
                  This survey will take about two minutes of your time.  
                  If you are eligible to take part in our study, our team will reach out shortly.'''),
                  question_description = 'welcome')
)

# ---- satisfied
survey_q.append(
    create_multiple_choice(question_number = 2, 
                           question_text = markdown('''How satisfied are you with Spotify?'''),
                           question_description = 'satisfaction',
                           values = [-2,-1,0,1,2], 
                           labels = ['very_dis', 'some_dis', 'neither', 'some_satis', 'very_satis'], 
                           display_text = [markdown(i) for i in ['Very dissatisfied',
                                                                 'Somewhat dissatisfied',
                                                                 'Neither satisfied or dissatisfied',
                                                                 'Somewhat satisfied',
                                                                 'Very satisfied']],
                           force_response = False,
                           multiple_answer = False, 
                           align = 'vertical')
)

# ---- devices
survey_q.append(
    create_multiple_choice(question_number = 3,
                           question_text = markdown('''In the past week, which device(s) were you using?  
                           Please select all that apply.'''),
                           question_description = 'devices',
                           values = None, 
                           labels = ['mobile', 'tablet', 'computer', 'game_console', 'tv', 'car', 'smart_speaker', 'other'], 
                           display_text = [markdown(i) for i in ['Mobile Phone', 'Tablet', 'Computer', 
                                                                 'Game console', 'TV', 'Car', 
                                                                 'Smart Speaker (e.g., Amazon Echo, Google Home)',
                                                                 'Other (please specify)']],
                           force_response = False,
                           multiple_answer = True, 
                           align = 'column')
)

# ---- age
survey_q.append(
    create_multiple_choice(question_number = 4, 
                           question_text = markdown('''How old are you?'''),
                           question_description = 'age',
                           values = [17,18,25,30,35,40,45,50,56], 
                           labels = ['under_18','18_24','25_29','30_34','35_39','40_44','45_49','50_55','55_plus'], 
                           display_text = [markdown(i) for i in ['**Under 18** years old', 
                                                                 '**18-24** years old', 
                                                                 '**25-29** years old', 
                                                                 '**30-34** years old', 
                                                                 '**35-39** years old', 
                                                                 '**40-44** years old', 
                                                                 '**45-49** years old', 
                                                                 '**50-55** years old', 
                                                                 '**55+** years old']], 
                           force_response = True,
                           multiple_answer = False, 
                           align = 'vertical') # vertical, horizontal, column, dropdown
)

# ---- employment
survey_q.append(
    create_multiple_choice(question_number = 5, 
                           question_text = markdown('''What is your current employment status?'''),
                           question_description = 'employment',
                           values = None, 
                           labels = ['student', 'stay_at_home', 'unemployed_retired',
                                     'self_consultant', 'self_owner',
                                     'part_time', 'full_time'], 
                           display_text = [markdown(i) for i in ['Studying full or part time', 'Stay-at-home parent/guardian', 'Unemployed/Retired',
                                                                 'Self-employed - consultant/freelancer', 'Self-employed - run my own business/company',
                                                                 'Part-time employment', 'Full-time employment']], 
                           force_response = False,
                           multiple_answer = False, 
                           align = 'dropdown') # vertical, horizontal, column, dropdown
)

# ---- frustration
survey_q.append(
    create_text_entry(question_number = 6, 
                      question_text = markdown('''When you use Spotify, what do you find frustrating?'''),
                      question_description = 'frustrating_open',
                      force_response = False)
)

# --------------------------------------------------------------------------------
# ---- post questions to survey --------------------------------------------------
# --------------------------------------------------------------------------------
for q in survey_q:
    post_questions_to_survey(qualtrics_ids, survey = survey, question = q, block_id = None)
    
# pool = mp.Pool()
# pool.starmap(post_questions_to_survey, [(qualtrics_ids, survey, q, None) for q in survey_q])
    
# --------------------------------------------------------------------------------
# ---- add embedded data fields --------------------------------------------------
# --------------------------------------------------------------------------------
add_embedded_data_fields_from_table(qualtrics_ids,
                                    dataframe = user_data.loc[:,'market':],
                                    survey = survey)

Embedded data fields added to recruiting_built_from_api


In [5]:
response = json.loads(requests.get(
        'https://{}/API/v3//survey-definitions/{}'.format(
            qualtrics_ids['data_center'], survey['id']
            ),
        headers={
            'x-api-token': qualtrics_ids['api_token'],
            'content-type': 'application/json'
            }
        ).text)

In [6]:
response['result']['Questions']['QID1']

{'QuestionID': 'QID1',
 'DataExportTag': 'Q3',
 'QuestionText': '<p>In the past week, which device(s) were you using?<br />\n                           Please select all that apply.</p>',
 'QuestionDescription': 'devices',
 'QuestionType': 'MC',
 'Selector': 'MACOL',
 'Choices': {'1': {'Display': '<p>Mobile Phone</p>'},
  '2': {'Display': '<p>Tablet</p>'},
  '3': {'Display': '<p>Computer</p>'},
  '4': {'Display': '<p>Game console</p>'},
  '5': {'Display': '<p>TV</p>'},
  '6': {'Display': '<p>Car</p>'},
  '7': {'Display': '<p>Smart Speaker (e.g., Amazon Echo, Google Home)</p>'},
  '8': {'Display': '<p>Other (please specify)</p>', 'TextEntry': 'true'}},
 'ChoiceOrder': [1, 2, 3, 4, 5, 6, 7, 8],
 'VariableNaming': {'1': 'mobile',
  '2': 'tablet',
  '3': 'computer',
  '4': 'game_console',
  '5': 'tv',
  '6': 'car',
  '7': 'smart_speaker',
  '8': 'other'},
 'Language': [],
 'Configuration': {'QuestionDescriptionOption': 'SpecifyLabel'},
 'QuestionText_Unsafe': '<p>In the past week, which de

---
---
---

# Create the unique survey links
When a person opens the survey using their unique link, their user data gets embedded.

In [66]:
# create unique survey links; when a user opens the survey link, it embeds their user data into the survey response
unique_links = []
for row in user_data.index:
    # ---- data needs to be base64 encoded
    encoded_user_data = base64.b64encode(bytes(user_data.loc[row].to_json(), 'utf-8')).decode('utf-8')
    # ---- encoded data gets appended to the end of the survey link using Qualtric's Q_EED function
    unique_links.append('https://{}/jfe/form/{}?Q_EED={}'.format(data_center, recruiting_survey, encoded_user_data))
user_data['unique_survey_link'] = unique_links

# ---- export survey distribution list
email_distribution_list = user_data.set_index('email')[['unique_survey_link']]
email_distribution_list.to_excel('recruiting_survey_distribution.xlsx')

# ---- show top 5 lines of email distribution list
email_distribution_list.head()

,unique_survey_link
email,
example_email0@outlook.com,https://duke.ca1.qualtrics.com/jfe/form/SV_2h7...
example_email1@icloud.com,https://duke.ca1.qualtrics.com/jfe/form/SV_2h7...
example_email2@outlook.com,https://duke.ca1.qualtrics.com/jfe/form/SV_2h7...
example_email3@icloud.com,https://duke.ca1.qualtrics.com/jfe/form/SV_2h7...
example_email4@gmail.com,https://duke.ca1.qualtrics.com/jfe/form/SV_2h7...


# Download Data

In [68]:
data = download_data(recruiting_survey, data_center, api_token)

response objects pulled...
column dictionary created...
survey responses formatted...


# Mailing List
Body can also include `firstName`, `lastName`, `phone`, `language` etc.

In [74]:
# ---- get values from recruiting survey
mailing_list = data['values']

# ---- name of the recruiting survey
survey_name = 'recruiting_survey_id'

# ---- columns that we want to embed in the mailing list
columns_to_embed = ['device_type', 'account_type', 'searches_in_app_30_days']

# ---- create an embeddedData column to match format required by Qualtrics
mailing_list = mailing_list[mailing_list.market == 'US'][['email']+columns_to_embed].reset_index().rename(columns={'index':survey_name})
mailing_list['embeddedData'] = [json.loads(mailing_list[[survey_name] + columns_to_embed].loc[idx].to_json()) for idx in mailing_list.index]
mailing_list = mailing_list[['email', 'embeddedData']]

In [22]:
response = requests.post('https://{}/API/v3/mailinglists'.format(data_center),
                         data=pd.Series(['mailing_list', library_id, 'qualtrics_functions'], index=['name', 'libraryId', 'category']).to_json(), 
                         headers={'x-api-token': api_token, 'content-type': 'application/json'})

if pd.read_json(response.text).loc['httpStatus', 'meta'] == '200 - OK':
    mailing_list_id = pd.read_json(response.text).loc['id', 'result']
    print('mailing list created')
else:
    print('error...')
    print(response.text)

mailing list created


In [77]:
for idx in mailing_list.index:
    response = requests.post('https://{}/API/v3/mailinglists/{}/contacts'.format(data_center, mailing_list_id),
                             data=mailing_list.loc[idx].to_json(), 
                             headers={'x-api-token': api_token, 'content-type': 'application/json'})
    
    # ---- reporting on whether it worked
    if pd.read_json(response.text).loc['httpStatus', 'meta'] == '200 - OK':
        print('{} added to mailing list'.format(mailing_list.loc[idx, 'email']))
    else:
        print('error... adding {} to mailing list'.format(mailing_list.loc[idx, 'email']))
        print(response.text)

example_email1@icloud.com added to mailing list
example_email2@outlook.com added to mailing list
example_email4@gmail.com added to mailing list
example_email5@gmail.com added to mailing list
example_email6@gmail.com added to mailing list
example_email8@icloud.com added to mailing list
example_email9@icloud.com added to mailing list
